In [ ]:
import sys
sys.path.append('../assets/')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Main libraries
from data_shuffling_split import *
from preprocess_text import *
from datasets import load_dataset, list_datasets, Dataset, DatasetDict, ClassLabel
from transformers import AutoModel, AutoTokenizer
import matplotlib.pyplot as plt
import torch

In [ ]:
dialect_datasets = load_dataset('Abdelrahman-Rezk/Arabic_Dialect_Identification', use_auth_token=True)
dialect_datasets

In [ ]:
train_data = dialect_datasets['train']
print(train_data.column_names)
print("="*50)
print(train_data[0])
print("="*50)
print(train_data.features)
print("="*50)
print(train_data[:3])

In [ ]:
train_data.set_format(type="pandas")
df = train_data[:]
df.head()

In [ ]:
def label_int2str(row):
    return train_data.features['label'].int2str(row)
df['label_name'] = df['label'].apply(label_int2str)
df.head()

In [ ]:
train_data

In [ ]:
df['label_name'].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

In [ ]:
df['tokens per tweet'] = df['text'].str.split().apply(len)
df.boxplot("tokens per tweet", by="label_name", grid=False, showfliers=False, color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
train_data.reset_format()

## From Text to Tokens

- تحديد انهى موديلز هنستخدمها
- تحديد التكوينزر الخاص بكل موديل
- التجريب على بعض الأمثلة من الداتا

https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-da

https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa

https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa-did-madar-twitter5?text=%D8%B9%D8%A7%D9%85%D9%84+%D8%A7%D9%8A%D9%87+%D8%9F

Edit tokenizer in our model to be limited to 124 word !?

## Subword Tokenization
The basic idea behind subword tokenization is to combine the best aspects of character and word tokenization. On the one hand, we want to split rare words into smaller units to allow the model to deal with complex words and misspellings.

In [ ]:
model_ckpt = "CAMeL-Lab/bert-base-arabic-camelbert-msa"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
some_text = train_data['text'][8]
some_text

In [ ]:
encoded_text = tokenizer(some_text)
print(encoded_text)

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoded_text['input_ids']))
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded_text['input_ids'])))

In [ ]:
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)
print(tokenizer.model_input_names)

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

print(tokenize(train_data[:2]))

In [ ]:
batch = {}
batch['text'] = "الهدف من الحياة هو سث[MASK] . "
encoded_text = tokenize(batch)
print(encoded_text)
print(tokenizer.convert_ids_to_tokens(encoded_text['input_ids']))

### Model Special Token & ITs ids

In [ ]:
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.mask_token, tokenizer.mask_token_id)

In [ ]:
# batch_size=None to get the length inputs based on the large tweet
dialect_datasets_encoded = dialect_datasets.map(tokenize, batched=True, batch_size=512)

In [ ]:
print(dialect_datasets_encoded.column_names)

In [ ]:
# The AutoModel class converts the token encodings to embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)